In [ ]:
from ABM_MeanField_Cells.model import GentModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec


import colorcet as cc


np.set_printoptions(linewidth=400)

%matplotlib inline

import os
#set a nice seaborn style


In [3]:
sns.set()
sns.set(rc={'figure.figsize':(16,9)})
sns.set_style("whitegrid")
sns.set_context("talk")


In [4]:
SAVEDATA = True


In [5]:
termination_condition = True
starting_deployment = "centre_segr"

In [ ]:
n_steps = 100
num_agents = 2**12
#initialize the model
model = GentModel(num_agents= num_agents, width=7, height = 7, mode = "improve", starting_deployment =starting_deployment, 
                  p_g = 0.05, h = 10, 
                  empty_border=0, seed = 67, termination_condition=termination_condition)

In [ ]:
model.capacity

In [ ]:
model.presence_matrix

In [ ]:
presence = np.zeros((model.width, model.height))
for cell in model.grid.coord_iter():
    presence[cell[1]] = len(cell[0])

fig, ax = plt.subplots()
sns.heatmap(presence, cmap="Purples")
plt.xlabel("X", fontsize=38)
plt.ylabel("Y", fontsize=38)
#titl
plt.title("Number of agents per cell", fontsize=38)


In [11]:
palette = sns.color_palette(cc.glasbey, n_colors=model.width*model.height)

In [ ]:
from matplotlib.colors import LogNorm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Data for the histogram plot
incomes = [agent.wealth for agent in model.schedule.agents]
types = [agent.tipo for agent in model.schedule.agents]

incomes_agents = list(zip(incomes, types))
incomes_agents.sort(reverse=True)
incomes, types = zip(*incomes_agents)

def millions_formatter(x, pos):
    return f'{x / 1_000_000}M'

# Function to format x-axis labels in thousands (k)
def thousands_formatter(x, pos):
    return f'{int(x / 1_000)}k'

# Data for the heatmap plot
avgs = np.zeros((model.width, model.height))

for i in range(model.width):
    for j in range(model.height):
        cell_content = model.grid.get_cell_list_contents((i, j))
        avg_wealth = np.median([agent.wealth for agent in cell_content])
        avgs[i, j] = avg_wealth

# Create subplots with a squared figsize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), gridspec_kw={'width_ratios': [1, 1]})

# First subplot (now on the left): Heatmap of the median incomes
sns.heatmap(avgs, cmap="rocket_r", square=True, ax=ax1, 
            cbar_kws={'label': 'Wealth', 'orientation': 'horizontal', "shrink": 0.68, 'pad': 0.12}, 
            linewidths=3, norm=LogNorm(vmin=avgs.min(), vmax=avgs.max()))

# Customize the color bar
# Customize the color bar
cbar = ax1.collections[0].colorbar
cbar.set_label('Median Income', fontsize=20, weight='bold')
# Move the label above the color bar
cbar.ax.xaxis.label.set_position((0.5, 1.2))  # Adjust position above the color bar
cbar.ax.xaxis.set_label_coords(0.5, 2.6)  # Fine-tune label position above the bar
# Adjust ticks for the horizontal color bar
cbar.ax.tick_params(labelsize=24, pad=12)  # Adjust tick size and padding
cbar.ax.xaxis.set_major_formatter(FuncFormatter(thousands_formatter))  # Format ticks as thousands


ax1.set_xticks([])
ax1.set_yticks([])

for spine in ax1.spines.values():
    spine.set_visible(False)

# Second subplot (now on the right): Plotting the incomes with colored markers based on agent type
indices = np.arange(len(incomes))

# Convert incomes and types to NumPy arrays for masking
incomes = np.array(incomes)
types = np.array(types)

# Color and label mappings
color_map = {'A': '#2a4b7a', 'B': '#00957c', 'C': '#f54333'}
label_map = {'A': 'H', 'B': 'M', 'C': 'L'}

# Plot all points for each agent type in a single command
for agent_type in ['A', 'B', 'C']:
    mask = (types == agent_type)
    marker = '*' if agent_type == 'A' else 'o'
    marker_size = 16 if agent_type == 'A' else 14
    ax2.plot(indices[mask], incomes[mask], marker, markersize=marker_size, alpha=1,
             color=color_map[agent_type], linestyle='None', label=label_map[agent_type])

# Remove duplicate labels (if any)
handles, labels = ax2.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax2.legend(by_label.values(), by_label.keys(), fontsize=28, title='Agent Type', title_fontsize=30, loc='upper right')

ax2.set_xlabel("Agents (ranked)", fontsize=34)
ax2.set_ylabel("Income (USD)", fontsize=34)
ax2.tick_params(axis='x', labelsize=28)
#set xticks every 4000
ax2.set_xticks(np.arange(0, len(incomes), 4000))
ax2.tick_params(axis='y', labelsize=28)
ax2.yaxis.set_major_formatter(FuncFormatter(millions_formatter))

# Add grid and customize spines for the second plot
ax2.grid(alpha=0.5, linestyle='--')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_linewidth(3)
ax2.spines['bottom'].set_linewidth(3)

ax2.spines['left'].set_color('black')
ax2.spines['bottom'].set_color('black')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(wspace=0.3)

# Add the labels 'a)' and 'b)' using figure coordinates
fig.canvas.draw()  # Necessary to update the positions

# Get the positions of the axes in figure coordinates
ax1_pos = ax1.get_position()
ax2_pos = ax2.get_position()

# Use the same y-coordinate for both labels
label_y = ax1_pos.y1 + 0.03  # Slightly above the top of ax1

# Place the labels using figure coordinates
fig.text(ax1_pos.x0 - 0.05, label_y, 'a)', fontsize=30, fontweight='bold', va='bottom', ha='left')
fig.text(ax2_pos.x0 - 0.1, label_y, 'b)', fontsize=30, fontweight='bold', va='bottom', ha='left')

# Save with tight layout
if SAVEDATA:
     plt.savefig("income_ranked_heatmap_new.pdf", format='pdf', bbox_inches='tight')

# Show the plots
plt.show()

In [ ]:
#count A, B and C in types array

A = types.count("A")
B = types.count("B")
C = types.count("C")

print("A: ", A, "B: ", B, "C: ", C)

In [ ]:
model.run_model(n_steps)

#while model.running and model.schedule.steps < n_steps:
#    model.step()

In [ ]:
#get the df of the datacollector
df = model.datacollector.get_model_vars_dataframe()
df_agents = model.datacollector.get_agent_vars_dataframe()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
if SAVEDATA:

    if termination_condition:
        to_path = f"out/termination/{starting_deployment}/"
        os.makedirs(to_path, exist_ok=True)
    else:
        to_path = f"out/long/{starting_deployment}/"
        os.makedirs(to_path, exist_ok=True)

    print(to_path)
    df.to_csv(f"{to_path}/results_model.csv")
    df_agents.to_csv(f"{to_path}/results_agents.csv")

In [ ]:
df_cells = df.copy()




df_cell_median = df_cells[["median_richness_matrix"]]
#transform the dictionary into a list
df_cell_median["median_richness_matrix"] = df_cell_median["median_richness_matrix"].apply(lambda x: list(x.flatten()))
df_cell_median = df_cell_median["median_richness_matrix"].apply(pd.Series)
df_cell_median.index = df_cell_median.index + 1



In [ ]:
sns.lineplot(data=df_cell_median, legend = False, palette = palette, linewidth=2.5, dashes=False, alpha = 0.8)

plt.title(str(model.num_agents) +" agents "   + str(model.p_g) + "p_g " + str(model.h) +"h" , fontsize=30)
plt.ylabel("Median Richness", fontsize=25)
plt.xlabel("Steps", fontsize=25)
#set xticks to appear every 10 steps
#plt.xticks(np.arange(0, n_steps+1, 10));



In [ ]:
df_cell_median = df_cell_median.drop(columns = 25)
sns.lineplot(data=df_cell_median, legend = False, palette = palette, linewidth=2.5, dashes=False, alpha = 0.8)

plt.title(str(model.num_agents) +" agents "   + str(model.p_g) + "p_g " + str(model.h) +"h" , fontsize=30)
plt.ylabel("Median Richness", fontsize=25)
plt.xlabel("Steps", fontsize=25)
#set xticks to appear every 10 steps
#plt.xticks(np.arange(0, n_steps+1, 10));



In [ ]:
df_cell_median_smooth = df_cell_median.copy()
df_cell_median_smooth = df_cell_median_smooth.rolling(model.h).mean()

sns.lineplot(data=df_cell_median_smooth, legend = False, palette = palette, linewidth=2.5, dashes=False, alpha = 0.8)

plt.title(str(model.num_agents) +" agents "   + str(model.p_g) + "p_g " + str(model.h) +"h" , fontsize=30)
plt.ylabel("Median Richness", fontsize=25)

plt.xlabel("Steps", fontsize=25)



In [ ]:
df_unhappy = df.copy()
df_unhappy = df_unhappy[["unhappy_C", "desire_to_move_C"]]


sns.lineplot(data=df_unhappy)
